# Clickbait Detection using BERT

Este notebook implementa la segunda parte de la práctica de detección de clickbait utilizando un modelo de lenguaje de gran tamaño (LLM) basado en BERT.

> **Nota importante sobre versiones**: Este notebook está diseñado para funcionar con versiones específicas de bibliotecas: transformers==4.18.0 y torch==1.11.0. Si experimentas errores relacionados con parámetros de TrainingArguments, asegúrate de que estás utilizando estas versiones.

> **Actualizaciones**: Se han implementado varias mejoras al modelo original, incluyendo diferentes esquemas de entrenamiento, modelos alternativos como RoBERTa, y técnicas de ajuste fino como LoRA.

In [ ]:
# Instalación de las bibliotecas necesarias
!pip install torch==1.11.0 transformers==4.18.0 scikit-learn pandas numpy matplotlib seaborn
!pip install peft==0.3.0  # Para implementar LoRA

In [ ]:
# Verificar versiones de las bibliotecas instaladas
import transformers
import torch
import pandas as pd
import sklearn
import peft

print(f"Transformers version: {transformers.__version__}")
print(f"PyTorch version: {torch.__version__ if torch is not None else 'Not installed'}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"PEFT version: {peft.__version__ if 'peft' in globals() else 'Not installed'}")

## 1. Importación de Bibliotecas

Importamos todas las bibliotecas necesarias para el procesamiento de datos, entrenamiento del modelo y evaluación.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import csv
import os
from collections import Counter

# Verificar si estamos en Google Colab y configurar GPU
try:
    import google.colab
    from google.colab import files
    IN_COLAB = True
    print("Ejecutando en Google Colab")
    
    # Verificar GPU disponible
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
        print(f"Número de GPUs disponibles: {torch.cuda.device_count()}")
    else:
        device = torch.device("cpu")
        print("No hay GPU disponible, usando CPU")
except:
    IN_COLAB = False
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"No estamos en Colab. Usando dispositivo: {device}")

## 2. Carga del Corpus

Cargamos los corpus para entrenamiento, validación y prueba:
- `TA1C_dataset_detection_train.csv` - Para entrenamiento original
- `TA1C_dataset_detection_dev.csv` - Para validación
- `TA1C_dataset_detection_dev_gold.csv` - Para evaluación con etiquetas de referencia

In [ ]:
# Configuración de rutas - ajustar según sea necesario para Google Colab
if IN_COLAB:
    print("Por favor, sube los archivos de datos:")
    uploaded = files.upload()  # Sube los archivos necesarios
    
    # Ruta de los archivos en Colab
    train_filepath = "TA1C_dataset_detection_train.csv"
    dev_filepath = "TA1C_dataset_detection_dev.csv"
    dev_gold_filepath = "TA1C_dataset_detection_dev_gold.csv"
    
    # Verificar que los archivos se han subido correctamente
    import os
    if not all(os.path.exists(f) for f in [train_filepath, dev_filepath, dev_gold_filepath]):
        print("ADVERTENCIA: No se han encontrado todos los archivos necesarios.")
        print("Por favor, asegúrate de subir los siguientes archivos:")
        print("- TA1C_dataset_detection_train.csv")
        print("- TA1C_dataset_detection_dev.csv")
        print("- TA1C_dataset_detection_dev_gold.csv")
else:
    # Rutas para entorno local
    train_filepath = "./corpus/TA1C_dataset_detection_train.csv"
    dev_filepath = "./corpus/TA1C_dataset_detection_dev.csv"
    dev_gold_filepath = "./corpus/TA1C_dataset_detection_dev_gold.csv"  # Archivo gold con etiquetas

# Función para cargar y mostrar información básica de un dataset
def load_and_display_dataset(filepath, name):
    try:
        data = pd.read_csv(filepath)
        print(f"Corpus {name} cargado con {len(data)} instancias")

        # Mostrar primeras filas
        print(f"\nPrimeras filas del corpus {name}:")
        display(data.head())

        # Información básica del corpus
        print(f"\nInformación del corpus {name}:")
        print(f"Columnas: {data.columns.tolist()}")
        
        # Verificar si existe la columna 'Tag Value'
        if 'Tag Value' in data.columns:
            print(f"Distribución de clases:")
            print(data["Tag Value"].value_counts())
            
        return data
    except FileNotFoundError as e:
        print(f"Error: No se encontró el archivo - {e}")
        print("\nSolución: Asegúrate de que los archivos están en las rutas correctas.")
        if IN_COLAB:
            print("En Google Colab, debes subir los archivos manualmente.")
        return None
    except Exception as e:
        print(f"Error inesperado: {e}")
        return None

# Cargar todos los datasets
train_data = load_and_display_dataset(train_filepath, "de entrenamiento")
dev_data = load_and_display_dataset(dev_filepath, "de desarrollo")
dev_gold_data = load_and_display_dataset(dev_gold_filepath, "gold de desarrollo")

# Verificar si se cargaron correctamente
has_gold = dev_gold_data is not None
if not has_gold:
    print("\nADVERTENCIA: No se pudo cargar el archivo gold de desarrollo.")
    print("Algunas funcionalidades de evaluación estarán limitadas.")

## 3. Implementación de Esquemas de Entrenamiento y Evaluación

Vamos a implementar los diferentes esquemas propuestos:

### Esquema 1
- 75% de TA1C_dataset_detection_train.csv para entrenamiento
- 25% de TA1C_dataset_detection_train.csv para evaluación
- TA1C_dataset_detection_dev_gold.csv para prueba

### Esquema 2
- 100% de TA1C_dataset_detection_train.csv para entrenamiento
- 100% de TA1C_dataset_detection_dev_gold.csv para evaluación

### Esquema 3
- 100% de TA1C_dataset_detection_train.csv para entrenamiento
- 50% de TA1C_dataset_detection_dev_gold.csv para evaluación
- 50% restante de TA1C_dataset_detection_dev_gold.csv para prueba

In [ ]:
# Función para preparar datos según el esquema
def prepare_data_for_scheme(scheme):
    """
    Prepara los datos según los diferentes esquemas de entrenamiento
    
    Args:
        scheme: Número del esquema (1, 2 o 3)
        
    Returns:
        Tupla con (X_train, y_train, X_val, y_val, X_test, y_test)
    """
    le = LabelEncoder()
    
    # Verificar que los datasets necesarios estén cargados
    if not train_data is not None:
        print("Error: No se ha podido cargar el archivo train.csv")
        return None, None, None, None, None, None, None, None
        
    if not has_gold:
        print("Error: No se ha podido cargar el archivo dev_gold.csv necesario para estos esquemas")
        return None, None, None, None, None, None, None, None
    
    if scheme == 1:
        print("\n=== Preparando datos para Esquema 1 ===")
        print("75% de train.csv para entrenamiento, 25% de train.csv para evaluación, dev_gold.csv para prueba")
        
        # Extraer características y etiquetas del archivo train
        X_train_data = train_data["Teaser Text"].tolist()
        y_train_data = train_data["Tag Value"].tolist()
        
        # Dividir en conjuntos de entrenamiento y validación (75% - 25%)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train_data, y_train_data, test_size=0.25, random_state=0, stratify=y_train_data, shuffle=True
        )
        
        # Usar dev_gold.csv como conjunto de prueba
        X_test = dev_data["Teaser Text"].tolist() if dev_data is not None else []
        y_test = dev_gold_data["Tag Value"].tolist()
            
    elif scheme == 2:
        print("\n=== Preparando datos para Esquema 2 ===")
        print("100% de train.csv para entrenamiento, 100% de dev_gold.csv para evaluación")
        
        # Usar train.csv como conjunto de entrenamiento
        X_train = train_data["Teaser Text"].tolist()
        y_train = train_data["Tag Value"].tolist()
        
        # Usar dev_gold.csv como conjunto de validación y prueba
        X_val = dev_data["Teaser Text"].tolist() if dev_data is not None else []
        X_test = X_val  # Mismo conjunto para validación y prueba
        y_val = dev_gold_data["Tag Value"].tolist()
        y_test = y_val
            
    elif scheme == 3:
        print("\n=== Preparando datos para Esquema 3 ===")
        print("100% de train.csv para entrenamiento, 50% de dev_gold.csv para evaluación, 50% restante para prueba")
        
        # Usar train.csv como conjunto de entrenamiento
        X_train = train_data["Teaser Text"].tolist()
        y_train = train_data["Tag Value"].tolist()
        
        # Dividir dev_gold.csv y sus etiquetas en dos mitades
        X_dev = dev_data["Teaser Text"].tolist() if dev_data is not None else []
        y_dev = dev_gold_data["Tag Value"].tolist()
        
        X_val, X_test, y_val, y_test = train_test_split(
            X_dev, y_dev, test_size=0.5, random_state=0, shuffle=True
        )
    else:
        raise ValueError(f"Esquema {scheme} no reconocido. Debe ser 1, 2 o 3")
        
    # Codificar etiquetas
    y_train_encoded = le.fit_transform(y_train)
    
    if y_val is not None:
        y_val_encoded = le.transform(y_val)
    else:
        y_val_encoded = None
        
    if y_test is not None:
        y_test_encoded = le.transform(y_test)
    else:
        y_test_encoded = None
        
    # Guardar mapeo de etiquetas
    label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print("\nMapeo de etiquetas:")
    for label, idx in label_mapping.items():
        print(f"  {label} -> {idx}")
    
    # Mostrar estadísticas
    print(f"\nConjunto de entrenamiento: {len(X_train)} instancias")
    print(f"Conjunto de validación: {len(X_val)} instancias")
    print(f"Conjunto de prueba: {len(X_test)} instancias")
    
    # Retornar todos los conjuntos y el mapeo de etiquetas
    return X_train, y_train_encoded, X_val, y_val_encoded, X_test, y_test_encoded, label_mapping, le

# Implementar cada esquema
data_schemes = {}
for scheme in [1, 2, 3]:
    data_schemes[scheme] = prepare_data_for_scheme(scheme)

In [ ]:
# Función para entrenar un modelo
def train_model(model_name, train_dataset, val_dataset, num_labels, output_dir, 
                learning_rate=2e-5, batch_size=16, num_epochs=3, use_lora=False, lora_config=None):
    """
    Entrena un modelo de transformers
    
    Args:
        model_name: Nombre del modelo preentrenado
        train_dataset, val_dataset: Datasets de entrenamiento y validación
        num_labels: Número de clases
        output_dir: Directorio para guardar resultados
        learning_rate, batch_size, num_epochs: Hiperparámetros
        use_lora: Si se debe usar LoRA para el ajuste fino
        lora_config: Configuración específica para LoRA
        
    Returns:
        trainer: Objeto Trainer con el modelo entrenado
        results: Resultados de la evaluación
    """
    # Determinar tipo de modelo
    model_type = "roberta" if "roberta" in model_name.lower() else "bert"
    
    # Cargar modelo preentrenado según tipo
    if model_type == "roberta":
        model = RobertaForSequenceClassification.from_pretrained(
            model_name, 
            num_labels=num_labels
        )
    else:
        model = BertForSequenceClassification.from_pretrained(
            model_name, 
            num_labels=num_labels
        )
    
    # Aplicar LoRA si se especifica
    if use_lora:
        # Configuración de LoRA predeterminada si no se proporciona
        if lora_config is None:
            lora_config = LoraConfig(
                task_type=TaskType.SEQ_CLS,
                r=8,  # Rango de la adaptación
                lora_alpha=16,
                lora_dropout=0.1,
                bias="none",
                target_modules=["query", "key", "value"]  # Módulos a adaptar
            )
        
        # Aplicar LoRA al modelo
        model = get_peft_model(model, lora_config)
        print("Información sobre parámetros entrenables con LoRA:")
        model.print_trainable_parameters()
    
    # Configurar argumentos de entrenamiento
    training_args = TrainingArguments(
        output_dir=output_dir,
        eval_strategy="steps",  # Compatible con transformers 4.18.0
        eval_steps=50,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        seed=0,
        load_best_model_at_end=True,
        save_strategy="steps",
        logging_steps=10,
        fp16=torch.cuda.is_available(),
        report_to="none"  # Deshabilitar informes a Weights & Biases
    )

    # Inicializar el entrenador
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Entrenar el modelo
    try:
        print(f"Entrenando modelo {model_name} {'con LoRA' if use_lora else 'sin LoRA'}...")
        trainer.train()
    except Exception as e:
        print(f"Error durante el entrenamiento: {str(e)}")
        print("\nTIP: Si el error está relacionado con 'evaluation_strategy', asegúrate de estar usando transformers==4.18.0")
        try:
            # Intenta modificar el argumento según la versión
            if hasattr(TrainingArguments, "eval_strategy"):
                training_args.evaluation_strategy = None
                training_args.eval_strategy = "steps"
            else:
                training_args.eval_strategy = None
                training_args.evaluation_strategy = "steps"
            
            # Reinicializar el entrenador con los argumentos corregidos
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                compute_metrics=compute_metrics
            )
            trainer.train()
        except Exception as e2:
            print(f"Error durante el segundo intento: {str(e2)}")
            raise

    # Evaluar el modelo
    print("\nEvaluando modelo...")
    results = trainer.evaluate()
    print(f"Resultados de la evaluación: {results}")
    
    return trainer, results

In [ ]:
# Ejecutar experimentos con diferentes configuraciones
def run_experiments(scheme_id, X_train, y_train, X_val, y_val, label_mapping, model_configs):
    """
    Ejecuta varios experimentos con diferentes modelos y configuraciones
    
    Args:
        scheme_id: ID del esquema de datos usado
        X_train, y_train, X_val, y_val: Datos preparados
        label_mapping: Mapeo de etiquetas
        model_configs: Lista de configuraciones de modelos a probar
        
    Returns:
        results_df: DataFrame con los resultados
    """
    results = []
    trainers = {}
    
    # Obtener número de clases
    num_labels = len(label_mapping) if label_mapping else 2
    
    for config in model_configs:
        model_name = config['model_name']
        use_lora = config.get('use_lora', False)
        lora_params = config.get('lora_params', None)
        
        print(f"\n{'=' * 80}")
        print(f"Preparando experimento con modelo: {model_name} {'con LoRA' if use_lora else 'sin LoRA'}")
        print(f"{'=' * 80}")
        
        # Inicializar tokenizador según tipo de modelo
        if "roberta" in model_name.lower():
            tokenizer = RobertaTokenizer.from_pretrained(model_name)
        else:
            tokenizer = BertTokenizer.from_pretrained(model_name)
        
        # Preparar datasets
        train_dataset, val_dataset = prepare_datasets(X_train, y_train, X_val, y_val, tokenizer)
        
        # Configurar LoRA si es necesario
        lora_config = None
        if use_lora and lora_params:
            lora_config = LoraConfig(
                task_type=TaskType.SEQ_CLS,
                r=lora_params.get('r', 8),
                lora_alpha=lora_params.get('lora_alpha', 16),
                lora_dropout=lora_params.get('lora_dropout', 0.1),
                bias=lora_params.get('bias', "none"),
                target_modules=lora_params.get('target_modules', ["query", "key", "value"])
            )
        
        # Nombre único para el experimento
        exp_name = f"{model_name.split('/')[-1]}_{config.get('name', '')}"
        if use_lora:
            exp_name += "_lora"
            
        try:
            print(f"\nEntrenando modelo: {exp_name}")
            
            # Entrenar modelo
            trainer, eval_results = train_model(
                model_name=model_name,
                train_dataset=train_dataset,
                val_dataset=val_dataset,
                num_labels=num_labels,
                output_dir=f"./results/scheme{scheme_id}_{exp_name}",
                learning_rate=config.get('learning_rate', 2e-5),
                batch_size=config.get('batch_size', 16),
                num_epochs=config.get('num_epochs', 3),
                use_lora=use_lora,
                lora_config=lora_config
            )
            
            # Guardar resultados
            results.append({
                'scheme': scheme_id,
                'model': model_name,
                'config': exp_name,
                'lora': 'Yes' if use_lora else 'No',
                'f1_macro': eval_results['eval_f1'],
                'precision': eval_results['eval_precision'],
                'recall': eval_results['eval_recall'],
                'accuracy': eval_results['eval_accuracy']
            })
            
            # Guardar el trainer para usar en predicciones
            trainers[exp_name] = (trainer, tokenizer)
            
        except Exception as e:
            print(f"Error en el experimento {exp_name}: {str(e)}")
            continue
    
    # Crear DataFrame con resultados
    results_df = pd.DataFrame(results)
    
    # Mostrar resultados
    print("\nResultados de los experimentos:")
    display(results_df)
    
    return results_df, trainers

In [ ]:
# Función para realizar y evaluar predicciones
def predict_and_evaluate(trainer, X_test, y_test, tokenizer, label_encoder, model_name):
    """
    Realiza predicciones y evalúa el rendimiento
    
    Args:
        trainer: Modelo entrenado
        X_test, y_test: Datos de prueba
        tokenizer: Tokenizador para los datos
        label_encoder: Codificador de etiquetas
        model_name: Nombre del modelo para reportes
        
    Returns:
        report: Informe de clasificación
        predictions: Etiquetas predichas
    """
    # Preparar dataset de prueba
    test_dataset = prepare_test_dataset(X_test, tokenizer)
    
    # Realizar predicciones
    print(f"\nGenerando predicciones con modelo {model_name}...")
    predictions = trainer.predict(test_dataset)
    pred_classes = predictions.predictions.argmax(-1)
    
    # Convertir predicciones a etiquetas
    pred_labels = label_encoder.inverse_transform(pred_classes)
    
    # Si tenemos etiquetas para el test, evaluar rendimiento
    if y_test is not None:
        # Revertir la codificación de etiquetas para el informe
        true_labels = label_encoder.inverse_transform(y_test)
        
        # Generar informe de clasificación
        report = classification_report(true_labels, pred_labels, output_dict=True)
        report_str = classification_report(true_labels, pred_labels)
        print("\nInforme de clasificación:")
        print(report_str)
        
        # Generar matriz de confusión
        plt.figure(figsize=(10, 8))
        cm = confusion_matrix(y_test, pred_classes)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
        disp.plot(cmap="Blues")
        plt.title(f"Matriz de Confusión - {model_name}")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
        
        # Generar matriz de confusión normalizada
        plt.figure(figsize=(10, 8))
        cm_norm = confusion_matrix(y_test, pred_classes, normalize='true')
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=label_encoder.classes_)
        disp.plot(cmap="Blues", values_format=".2f")
        plt.title(f"Matriz de Confusión Normalizada - {model_name}")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
        
        return report, pred_labels
    else:
        print("No hay etiquetas disponibles para evaluar el rendimiento en el conjunto de prueba")
        return None, pred_labels

# Función para evaluar predicciones con todas las instancias de dev_gold
def evaluate_on_complete_dev_gold(trainer, tokenizer, label_encoder, model_name):
    """
    Evalúa el modelo en el conjunto completo de dev_gold
    
    Args:
        trainer: Modelo entrenado
        tokenizer: Tokenizador para los datos
        label_encoder: Codificador de etiquetas
        model_name: Nombre del modelo para reportes
    
    Returns:
        report: Informe de clasificación
    """
    if dev_gold_data is None or dev_data is None:
        print("No se pueden realizar predicciones en dev_gold: datos no disponibles")
        return None
    
    X_complete = dev_data["Teaser Text"].tolist()
    y_complete = dev_gold_data["Tag Value"].tolist()
    
    # Codificar etiquetas con el mismo codificador
    y_complete_encoded = label_encoder.transform(y_complete)
    
    print(f"\n{'='*80}")
    print(f"Evaluando modelo {model_name} en el conjunto COMPLETO de dev_gold ({len(X_complete)} instancias)")
    print(f"{'='*80}")
    
    report, _ = predict_and_evaluate(
        trainer=trainer,
        X_test=X_complete,
        y_test=y_complete_encoded,
        tokenizer=tokenizer,
        label_encoder=label_encoder,
        model_name=f"{model_name} (Dev Gold Completo)"
    )
    
    return report

In [ ]:
# Definir configuraciones de modelos a utilizar
model_configs = [
    # BERT base sin LoRA
    {
        'model_name': "dccuchile/bert-base-spanish-wwm-cased",
        'name': 'base',
        'learning_rate': 2e-5,
        'batch_size': 16,
        'num_epochs': 3,
        'use_lora': False
    },
    # BERT base con LoRA
    {
        'model_name': "dccuchile/bert-base-spanish-wwm-cased",
        'name': 'base',
        'learning_rate': 2e-5,
        'batch_size': 16,
        'num_epochs': 3,
        'use_lora': True,
        'lora_params': {
            'r': 8,
            'lora_alpha': 16,
            'lora_dropout': 0.1,
            'target_modules': ["query", "key", "value"]
        }
    },
    # RoBERTa base sin LoRA
    {
        'model_name': "PlanTL-GOB-ES/roberta-base-bne",
        'name': 'base',
        'learning_rate': 2e-5,
        'batch_size': 16,
        'num_epochs': 3,
        'use_lora': False
    },
    # RoBERTa base con LoRA
    {
        'model_name': "PlanTL-GOB-ES/roberta-base-bne",
        'name': 'base',
        'learning_rate': 2e-5,
        'batch_size': 16,
        'num_epochs': 3,
        'use_lora': True,
        'lora_params': {
            'r': 8,
            'lora_alpha': 16,
            'lora_dropout': 0.1,
            'target_modules': ["query", "key", "value"]
        }
    },
    # RoBERTa con LoRA optimizado
    {
        'model_name': "PlanTL-GOB-ES/roberta-base-bne",
        'name': 'optimized',
        'learning_rate': 1e-4,
        'batch_size': 32,
        'num_epochs': 5,
        'use_lora': True,
        'lora_params': {
            'r': 16,  # Mayor rango para más capacidad
            'lora_alpha': 32,
            'lora_dropout': 0.2,
            'target_modules': ["query", "key", "value", "dense"]  # Incluir capas densas
        }
    }
]

In [ ]:
# Ejecutar experimentos para cada esquema
all_results = []
all_reports = {}
dev_gold_complete_reports = {}

for scheme_id in [1, 2, 3]:
    print(f"\n\n{'#'*80}")
    print(f"# EJECUTANDO EXPERIMENTOS PARA ESQUEMA {scheme_id}")
    print(f"{'#'*80}\n")
    
    # Obtener datos para este esquema
    X_train, y_train, X_val, y_val, X_test, y_test, label_mapping, label_encoder = data_schemes[scheme_id]
    
    # Verificar que tenemos datos válidos
    if X_train is None or y_train is None:
        print(f"No se pueden ejecutar experimentos para el esquema {scheme_id}: datos no disponibles")
        continue
    
    # Ejecutar experimentos
    results_df, trainers = run_experiments(
        scheme_id=scheme_id,
        X_train=X_train, 
        y_train=y_train, 
        X_val=X_val, 
        y_val=y_val,
        label_mapping=label_mapping,
        model_configs=model_configs
    )
    
    # Añadir resultados a la lista global
    all_results.append(results_df)
    
    # Para cada modelo entrenado, generar predicciones y evaluación
    for model_name, (trainer, tokenizer) in trainers.items():
        # Primero evaluamos en el conjunto de prueba del esquema actual
        if X_test is not None:
            report, _ = predict_and_evaluate(
                trainer=trainer,
                X_test=X_test,
                y_test=y_test,
                tokenizer=tokenizer,
                label_encoder=label_encoder,
                model_name=f"Esquema {scheme_id} - {model_name}"
            )
            
            if report:
                all_reports[f"Esquema {scheme_id} - {model_name}"] = report
        
        # Luego evaluamos en el conjunto completo de dev_gold
        if has_gold:
            dev_gold_report = evaluate_on_complete_dev_gold(
                trainer=trainer,
                tokenizer=tokenizer,
                label_encoder=label_encoder,
                model_name=f"Esquema {scheme_id} - {model_name}"
            )
            
            if dev_gold_report:
                dev_gold_complete_reports[f"Esquema {scheme_id} - {model_name}"] = dev_gold_report

# Combinar todos los resultados
if all_results:
    combined_results = pd.concat(all_results)
    combined_results = combined_results.sort_values(by='f1_macro', ascending=False)

    print("\n\nRESUMEN DE TODOS LOS EXPERIMENTOS (ordenados por F1-macro):")
    display(combined_results)

    # Guardar resultados
    combined_results.to_csv("experimentos_clickbait_resultados_completos.csv", index=False)

    # Identificar el mejor modelo
    if not combined_results.empty:
        best_model = combined_results.iloc[0]
        print(f"\nMEJOR MODELO: {best_model['config']} (Esquema {best_model['scheme']})")
        print(f"F1-macro: {best_model['f1_macro']:.4f}")
        print(f"Precisión: {best_model['precision']:.4f}")
        print(f"Recall: {best_model['recall']:.4f}")
        print(f"Exactitud: {best_model['accuracy']:.4f}")
        
        # Crear un archivo con el reporte completo del mejor modelo en dev_gold
        best_model_key = f"Esquema {best_model['scheme']} - {best_model['config']}"
        if best_model_key in dev_gold_complete_reports:
            best_report = dev_gold_complete_reports[best_model_key]
            
            # Convertir el informe a DataFrame para mejor visualización
            report_df = pd.DataFrame(best_report).T
            report_df = report_df.drop('support', axis=1, errors='ignore')
            
            print("\nINFORME DE CLASIFICACIÓN DEL MEJOR MODELO EN DEV_GOLD COMPLETO:")
            display(report_df)
            
            # Guardar informe
            report_df.to_csv("mejor_modelo_reporte_dev_gold.csv")
else:
    print("\n\nNo se pudieron obtener resultados de ningún experimento.")

## 8. Análisis Comparativo de Resultados

En esta sección realizamos un análisis detallado de los diferentes experimentos y sus resultados.

### Comparación de Esquemas de Entrenamiento

Hemos evaluado tres esquemas diferentes de división de datos:

1. **Esquema 1**: Uso de 75% del conjunto train para entrenamiento y 25% para validación, con evaluación final en dev_gold.
2. **Esquema 2**: Uso del 100% del conjunto train para entrenamiento y evaluación en todo el conjunto dev_gold.
3. **Esquema 3**: Uso del 100% del conjunto train para entrenamiento y división del conjunto dev_gold (50% validación, 50% prueba).

### Comparación de Modelos

Hemos comparado dos arquitecturas de modelos:

- **BERT** (dccuchile/bert-base-spanish-wwm-cased): Modelo base en español con tokenización sensible a mayúsculas/minúsculas.
- **RoBERTa** (PlanTL-GOB-ES/roberta-base-bne): Versión mejorada de BERT entrenada con el corpus de la Biblioteca Nacional de España.

### Impacto de LoRA

Hemos evaluado la efectividad de la técnica Low-Rank Adaptation (LoRA) para ajuste fino con diferentes configuraciones:
- LoRA básico: r=8, alpha=16, dropout=0.1
- LoRA optimizado: r=16, alpha=32, dropout=0.2, incluyendo capas densas

### Análisis por Clase

Analizamos el rendimiento específico por clase, con especial atención en:
- Precisión, recall y F1-score para la clase "Clickbait"
- Principales confusiones en la matriz de confusión
- Casos donde el modelo presenta mayor dificultad

### Conclusiones Principales

[Este espacio se completará automáticamente con los resultados de los experimentos]

## 9. Recomendaciones y Trabajo Futuro

Basándonos en los resultados obtenidos, podemos hacer las siguientes recomendaciones:

### Configuración Óptima

- **Modelo Recomendado**: [se completará con el mejor modelo]
- **Esquema de Entrenamiento**: [se completará con el mejor esquema]
- **Uso de LoRA**: [recomendación basada en resultados]

### Áreas de Mejora

1. **Balanceo de Clases**: Nuestros experimentos muestran que [observaciones sobre el desbalance]

2. **Ajuste de Hiperparámetros**: [recomendaciones específicas]

3. **Arquitectura del Modelo**: [observaciones sobre BERT vs RoBERTa]

### Trabajo Futuro

1. **Explorar modelos más grandes**: Modelos como RoBERTa-large o BETO podrían ofrecer mejores resultados.

2. **Técnicas avanzadas de regularización**: Para evitar el sobreajuste, se podrían explorar más técnicas.

3. **Análisis cualitativo de errores**: Un análisis detallado de los tweets mal clasificados podría proporcionar insights valiosos.

4. **Ensembles de modelos**: Combinar las predicciones de varios modelos podría mejorar la robustez del sistema.

In [ ]:
def compute_metrics(pred):
    """
    Calcula métricas de evaluación para las predicciones del modelo.
    
    Args:
        pred: Objeto de predicción de Hugging Face
        
    Returns:
        dict: Diccionario con métricas calculadas
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Calcular métricas
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Función para tokenizar y preparar datasets
def prepare_datasets(X_train, y_train, X_val, y_val, tokenizer, max_length=128):
    """
    Tokeniza textos y prepara datasets de PyTorch
    
    Args:
        X_train, y_train: Datos de entrenamiento
        X_val, y_val: Datos de validación
        tokenizer: Tokenizador de transformers
        max_length: Longitud máxima de secuencia
        
    Returns:
        train_dataset, val_dataset: Datasets listos para entrenamiento
    """
    # Tokenizar textos
    train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
    
    # Crear clase Dataset personalizada
    class ClickbaitDataset(Dataset):
        def __init__(self, encodings, labels=None):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            if self.labels is not None:
                item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.encodings.input_ids)
    
    # Crear dataset de entrenamiento
    train_dataset = ClickbaitDataset(train_encodings, y_train)
    
    # Crear dataset de validación si hay etiquetas
    if X_val is not None:
        val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=max_length)
        if y_val is not None:
            val_dataset = ClickbaitDataset(val_encodings, y_val)
        else:
            val_dataset = ClickbaitDataset(val_encodings)
    else:
        val_dataset = None
        
    return train_dataset, val_dataset

# Función para tokenizar datos de prueba
def prepare_test_dataset(X_test, tokenizer, max_length=128):
    """
    Tokeniza textos de prueba y prepara dataset
    
    Args:
        X_test: Datos de prueba
        tokenizer: Tokenizador de transformers
        max_length: Longitud máxima de secuencia
        
    Returns:
        test_dataset: Dataset listo para inferencia
    """
    class TestDataset(Dataset):
        def __init__(self, encodings):
            self.encodings = encodings

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            return item

        def __len__(self):
            return len(self.encodings.input_ids)
    
    test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)
    return TestDataset(test_encodings)